In [1]:
# setup
import pandas as pd 
import geemap
import ee
ee.Authenticate()
ee.Initialize(project="cloud-cover-421214")
# LIMIT = 10 # limit number of bounding boxes for development

In [1]:
# load bounding box feature data
df = pd.read_csv("./FRT_coords/tank_inventory.csv")
df = df.dropna(axis=0, how='any')

features = [
    ee.Feature(ee.Geometry.Rectangle(
        coords=[
            float(r["NW Coor (Long, Lat)"][1:-1].split(", ")[0]), # nw long 
            float(r["SE Coor (Long, Lat)"][1:-1].split(", ")[1]), # se lat 
            float(r["SE Coor (Long, Lat)"][1:-1].split(", ")[0]), # se long 
            float(r["NW Coor (Long, Lat)"][1:-1].split(", ")[1]) # nw lat
        ]), 
        {"state":r["State"], "county":r["County"], "id": r["ID"]}
    )
    # for index, r in df.head(LIMIT).iterrows()
    for index, r in df.iterrows()
]

fc = ee.FeatureCollection(features)

NameError: name 'pd' is not defined

In [50]:
# load cloudscore image data
start_date = "2015-07-01"
end_date = "2024-05-01"

cloudscore = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED").filterDate(start_date, end_date).filterBounds(fc)

In [51]:
# core map reduce computation 
reduced = cloudscore.map(lambda image: image.reduceRegions(fc, ee.Reducer.mean()).map(lambda f: f.set('timestamp', ee.Date(image.get('system:time_start'))))).flatten()

task = ee.batch.Export.table.toDrive(
    collection=reduced, 
    description="full_cloudscores",
    folder="cloudscore", 
    fileFormat="CSV"
)
task.start()

Above cell returns "reduced": feature collection where each feature is data per timestamp per area. So, N boxes over M days produce N*M features. 

cs = mean clouscore. 
  cs WILL be None if a particular box was not covered by the image

timestamp = time this data was collected 

size: O(N*M) features

In [55]:
# re run this cell to check task status
task.status()


{'state': 'RUNNING',
 'description': 'full_cloudscores',
 'priority': 100,
 'creation_timestamp_ms': 1715442998473,
 'update_timestamp_ms': 1715443305459,
 'start_timestamp_ms': 1715443005332,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'id': 'HDVOQEDCHJ7O4QTJEHUJ6GGJ',
 'name': 'projects/cloud-cover-421214/operations/HDVOQEDCHJ7O4QTJEHUJ6GGJ'}